In [1]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-12-04 00:52:23 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,658633,35.2,1454671,77.7,1227764,65.6
Vcells,1222301,9.4,8388608,64.0,1975161,15.1


In [4]:
path_competencia_02 <- '/home/vickydiliscia/buckets/b1/datasets/competencia_02_crudo.csv.gz'
path_competencia_03 <- '/home/vickydiliscia/buckets/b1/datasets/competencia_03_crudo.csv.gz'
path_functions <- '/home/vickydiliscia/dmeyf2025/src/churn/functions'

In [5]:
source(file.path(path_functions, "index.R"))
source_functions(path_functions)

Sourcing: add_divisiones.R

Sourcing: add_ratio.R

Sourcing: agregar_clase_ternaria.R

Sourcing: agregar_var_rf.R

Sourcing: aplicar_dataquality_zeros.R

Sourcing: apply_sum_specs_multi.R

Sourcing: audit_nullear.R

Sourcing: build_variables_rotas.R

Sourcing: crear_proporcion.R

Sourcing: deflactar_por_ipc.R

Sourcing: features_historicos_optimizados.R

Loading required package: Rcpp

Sourcing: features_historicos.R

Sourcing: flip.R

Sourcing: interpolar_faltantes.R

Sourcing: lags.R

Sourcing: normalizar_ctrx_quarter.R

Sourcing: print_utils.R

Sourcing: run_features_historicas.R

Sourcing: tendencias_y_mas.R

Sourcing: ver_meses_rotos.R

✅ Todas las funciones fueron cargadas correctamente.



In [7]:
paths <- c(path_competencia_02,
           path_competencia_03)

dataset <- rbindlist(
  lapply(paths, fread),
  use.names = TRUE,
  fill      = TRUE
)

dataset <- agregar_clase_ternaria(dataset)
setorder(dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, .(foto_mes, clase_ternaria)]
Sys.time()

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,645
201901,BAJA+2,729
201901,CONTINUA,122899
201902,BAJA+1,733
201902,BAJA+2,707
201902,CONTINUA,123961
201903,BAJA+1,708
201903,BAJA+2,751
201903,CONTINUA,124508


[1] "2025-12-04 00:58:04 UTC"

In [11]:
library(data.table)

#--------------------------------------------------------
# 1) Armar dfuture y ganancia
#--------------------------------------------------------

# Asumo que ya tenés: dataset y PARAM cargados
dfuture <- copy(dataset[ foto_mes %in% c(202107)])

# Ganancia tipo competencia (ajustala si cambian los montos)
dfuture[, ganancia := ifelse(clase_ternaria == "BAJA+2", 780000, -20000)]

#--------------------------------------------------------
# 2) Leer TODAS las predicciones de 4 directorios y ensamblar
#--------------------------------------------------------

# 🔧 CAMBIAR ESTOS NOMBRES POR LOS TUSYOS REALES
dirs_preds <- c(
  "/home/vickydiliscia/buckets/b1/exp/APO3012-202107_100271/predicciones_modelitos",  # ej: "predicciones_modelitos_APO1"
  "/home/vickydiliscia/buckets/b1/exp/APO3012-202107_250061/predicciones_modelitos",  # ej: "predicciones_modelitos_APO2"
  "/home/vickydiliscia/buckets/b1/exp/APO3002-202107_100271/predicciones_modelitos",
  "/home/vickydiliscia/buckets/b1/exp/APO3002-202107_250061/predicciones_modelitos",
  "/home/vickydiliscia/buckets/b1/exp/APO3020-202107_100271/predicciones_modelitos"

)

# Listar todos los archivos de los 4 directorios
archivos <- unlist(lapply(dirs_preds, function(d) {
  list.files(d, pattern = "\\.(txt|csv)$", full.names = TRUE)
}))

cat("Cantidad de archivos de predicción encontrados:", length(archivos), "\n")


Cantidad de archivos de predicción encontrados: 250 


In [10]:
# Leer cada archivo y normalizar columnas: numero_de_cliente, prob
lista_preds <- lapply(archivos, function(f) {
  dt <- fread(f)
  
  # Aseguro columna numero_de_cliente
  if (!"numero_de_cliente" %in% names(dt)) {
    setnames(dt, old = names(dt)[1L], new = "numero_de_cliente")
  }
  
  # Detecto la(s) columna(s) de probabilidad (todo lo que no sea numero_de_cliente)
  prob_cols <- setdiff(names(dt), "numero_de_cliente")
  if (length(prob_cols) != 1L) {
    stop("Archivo ", f, " no tiene exactamente una columna de prob además de numero_de_cliente")
  }
  
  setnames(dt, old = prob_cols, new = "prob")
  dt[, .(numero_de_cliente, prob)]
})

# Ensamble: promedio simple por cliente
preds_ensamble <- rbindlist(lista_preds)[
  , .(prob = mean(prob, na.rm = TRUE)), by = numero_de_cliente]

cat("Cantidad de clientes en el ensamble:", nrow(preds_ensamble), "\n")

#--------------------------------------------------------
# 3) Unir ensamble con dfuture y filtrar foto_mes == 202107
#--------------------------------------------------------

setkey(dfuture,         numero_de_cliente)
setkey(preds_ensamble,  numero_de_cliente)

df_ens <- dfuture[preds_ensamble, nomatch = 0]   # trae sólo clientes que tienen predicción

df_ens_202107 <- df_ens[ foto_mes == 202107 ]

cat("Filas para foto_mes 202107:", nrow(df_ens_202107), "\n")

#--------------------------------------------------------
# 4) Ordenar por prob, hacer ganancia acumulada
#    y evaluar cortes de 9000 a 12000
#--------------------------------------------------------

# Orden por probabilidad descendente
setorder(df_ens_202107, -prob)

# Ganancia acumulada
df_ens_202107[, gan_acum := cumsum(ganancia)]

# Cortes de 9000 a 12000
cortes <- seq(9000, 12000, by = 500)
                      
# Seguridad: evitar que el corte supere la cantidad de filas
cortes <- cortes[cortes <= nrow(df_ens_202107)]

res_cortes <- data.table(
  corte    = cortes,
  ganancia = df_ens_202107[cortes, gan_acum]
)

# Ver tabla de cortes y ganancias
print(res_cortes)

# Mejor corte en ese rango
best_en_rango <- res_cortes[ which.max(ganancia) ]
cat("\nMejor corte entre 9000 y 12000:\n")
print(best_en_rango)

#--------------------------------------------------------
# 5) (Opcional) Generar submit para un N elegido (ej: 11000)
#--------------------------------------------------------

N <- 11000L
N <- min(N, nrow(df_ens_202107))  # por las dudas

df_ens_202107[, Predicted := 0L]
df_ens_202107[1:N, Predicted := 1L]

# Ganancia explícita para ese N (chequeo alternativo)
gan_N <- df_ens_202107[
  , sum(ifelse(
    Predicted == 1 & clase_ternaria == "BAJA+2", 780000,
    ifelse(Predicted == 1 & clase_ternaria != "BAJA+2", -20000, 0)
  ))
]

cat("\nGanancia para N =", N, ":", gan_N, "\n")


Cantidad de clientes en el ensamble: 164596 
Filas para foto_mes 202107: 164596 
   corte  ganancia
   <num>     <num>
1:  9000 425600000
2:  9500 426000000
3: 10000 426400000
4: 10500 431600000
5: 11000 432800000
6: 11500 433200000
7: 12000 431200000

Mejor corte entre 9000 y 12000:
   corte  ganancia
   <num>     <num>
1: 11500 433200000

Ganancia para N = 11000 : 432800000 


In [12]:
# Leer cada archivo y normalizar columnas: numero_de_cliente, prob
lista_preds <- lapply(archivos, function(f) {
  dt <- fread(f)
  
  # Aseguro columna numero_de_cliente
  if (!"numero_de_cliente" %in% names(dt)) {
    setnames(dt, old = names(dt)[1L], new = "numero_de_cliente")
  }
  
  # Detecto la(s) columna(s) de probabilidad (todo lo que no sea numero_de_cliente)
  prob_cols <- setdiff(names(dt), "numero_de_cliente")
  if (length(prob_cols) != 1L) {
    stop("Archivo ", f, " no tiene exactamente una columna de prob además de numero_de_cliente")
  }
  
  setnames(dt, old = prob_cols, new = "prob")
  dt[, .(numero_de_cliente, prob)]
})

# Ensamble: promedio simple por cliente
preds_ensamble <- rbindlist(lista_preds)[
  , .(prob = mean(prob, na.rm = TRUE)), by = numero_de_cliente]

cat("Cantidad de clientes en el ensamble:", nrow(preds_ensamble), "\n")

#--------------------------------------------------------
# 3) Unir ensamble con dfuture y filtrar foto_mes == 202107
#--------------------------------------------------------

setkey(dfuture,         numero_de_cliente)
setkey(preds_ensamble,  numero_de_cliente)

df_ens <- dfuture[preds_ensamble, nomatch = 0]   # trae sólo clientes que tienen predicción

df_ens_202107 <- df_ens[ foto_mes == 202107 ]

cat("Filas para foto_mes 202107:", nrow(df_ens_202107), "\n")

#--------------------------------------------------------
# 4) Ordenar por prob, hacer ganancia acumulada
#    y evaluar cortes de 9000 a 12000
#--------------------------------------------------------

# Orden por probabilidad descendente
setorder(df_ens_202107, -prob)

# Ganancia acumulada
df_ens_202107[, gan_acum := cumsum(ganancia)]

# Cortes de 9000 a 12000
cortes <- seq(9000, 12000, by = 500)
                      
# Seguridad: evitar que el corte supere la cantidad de filas
cortes <- cortes[cortes <= nrow(df_ens_202107)]

res_cortes <- data.table(
  corte    = cortes,
  ganancia = df_ens_202107[cortes, gan_acum]
)

# Ver tabla de cortes y ganancias
print(res_cortes)

# Mejor corte en ese rango
best_en_rango <- res_cortes[ which.max(ganancia) ]
cat("\nMejor corte entre 9000 y 12000:\n")
print(best_en_rango)

#--------------------------------------------------------
# 5) (Opcional) Generar submit para un N elegido (ej: 11000)
#--------------------------------------------------------

N <- 11000L
N <- min(N, nrow(df_ens_202107))  # por las dudas

df_ens_202107[, Predicted := 0L]
df_ens_202107[1:N, Predicted := 1L]

# Ganancia explícita para ese N (chequeo alternativo)
gan_N <- df_ens_202107[
  , sum(ifelse(
    Predicted == 1 & clase_ternaria == "BAJA+2", 780000,
    ifelse(Predicted == 1 & clase_ternaria != "BAJA+2", -20000, 0)
  ))
]

cat("\nGanancia para N =", N, ":", gan_N, "\n")

Cantidad de clientes en el ensamble: 164596 
Filas para foto_mes 202107: 164596 
   corte  ganancia
   <num>     <num>
1:  9000 427200000
2:  9500 428400000
3: 10000 430400000
4: 10500 430800000
5: 11000 432000000
6: 11500 433200000
7: 12000 434400000

Mejor corte entre 9000 y 12000:
   corte  ganancia
   <num>     <num>
1: 12000 434400000

Ganancia para N = 11000 : 4.32e+08 


In [ ]:

# Submit sólo con numero_de_cliente (sin cabeceras)
fwrite(
  df_ens_202107[Predicted == 1, .(numero_de_cliente)],
  file      = sprintf("submit_ensamble_N%d.csv", N),
  col.names = FALSE
)
